# 2-1. 하스스톤 직업별 점유율 및 승률 통계 데이터 크롤링
<pre>
▶ 기간 : 2023년 11월 15일 ~ 11월 28일자 [최신 확장팩 "황야의 땅 결투" 출시일]
▶ 대상 : 전 서버에 존재하는 모든 등급 유저
▶ 사이트 : [하스스톤 메타](https://hsreplay.net/meta)
</pre>

In [32]:
import requests # HTTP 프로토콜을 이용하여 웹사이트로부터 데이터를 송수신 받기 위한 라이브러리
from bs4 import BeautifulSoup # 웹페이지의 HTML, XML 파일에서 데이터를 추출하는 라이브러리
import openpyxl # 엑셀파일 작업을 위한 라이브러리
import pandas as pd # 구조화된 데이터나 표 형식의 데이터 분석 및 조작을 위한 라이브러리

# 웹페이지를 자동으로 조작하기 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # 웹브라우저의 드라이버를 자동으로 관리하는 라이브러리

# Selenium에서 사용되는 웹요소를 찾기 위한 기능을 제공해주는 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyperclip # 자동 로그인을 위한 변수 복사 라이브러리
import re  #  정규표현식을 사용해 숫자를 추출하기 위한 라이브러리
from time import sleep  # 실행을 일시적으로 멈춰 로딩을 기다리기 위한 라이브러리

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


In [33]:
def create_excel() : 
  # 엑셀 만들기
  wb = openpyxl.Workbook()

  # 워크시트 만들기
  ws = wb.active
  ws.title = " 직업별 점유율"

  # 데이터 추가하기
  ws.append(['직업', '덱유형', '승률', '점유율', '게임수'])

  # 엑셀 저장하기
  wb.save('./hs_meta.xlsx')

In [34]:
def create_sheet(wb):
  # 기존 'Matchups' 시트가 존재하면 삭제
  if 'Matchups' in wb.sheetnames:
    wb.remove(wb['Matchups'])

  # 새로운 시트 생성하고 반환
  ws = wb.create_sheet(title="매치업")
  wb.save('./hs_meta.xlsx')
  
  return ws

In [35]:
def save_csv():
    # 엑셀 파일 직접 읽기
    df_matchups = pd.read_excel('./hs_meta.xlsx', sheet_name="매치업", skiprows=0, index_col=0)

    # CSV 파일로 저장
    df_matchups.to_csv('./hs_matchups.csv', encoding='utf-8-sig')

In [36]:
def wait_for_element(driver, by, value, timeout=10):
    try:
        element_present = EC.presence_of_element_located((by, value))
        WebDriverWait(driver, timeout).until(element_present)
    except Exception as e:
        print(f"엘리먼트 대기 중 예외 발생: {e}")
        driver.quit()

In [37]:
def get_user_input():
    # 입력값 받기
    time = int(input("검색할 기간을 입력하세요.\n[최근7일 : 7 // 최근 3일 : 3 // 최근 1일 : 1 // 최신 패치 : 0]"))
    rank = input("검색할 랭크를 입력하세요.\n[전구간 : A // 브론즈 : B // 실버 : S // 골드 : G // 플래티넘 : P // 다이아 : D // 전설 : L // 상위1000 : 1000]")
    server = input("검색할 서버를 입력하세요.\n[전서버 : A // 아시아 : AP // 유럽 : EU // 북미 : US]")

    # 검색 범위 딕셔너리 정의
    time_dict = {7: "LAST_7_DAYS", 3: "LAST_3_DAYS", 1: "LAST_1_DAY", 0: "ALL"}
    rank_dict = {"A": "ALL", "B": "BRONZE", "S": "SILVER", "G": "GOLD", "P": "PLATINUM", "D": "DIAMOND", "L": "LEGEND", "1000": "TOP_1000_LEGEND"}
    server_dict = {"A": "ALL", "AP": "REGION_KR", "EU": "REGION_EU", "US": "REGION_US"}

    # 입력값 확인
    if time not in time_dict or rank not in rank_dict or server not in server_dict:
        raise ValueError("올바르지 않은 입력값입니다.")

    return time_dict[time], rank_dict[rank], server_dict[server]

In [38]:
def driver_in(): 
# ChromeOptions 설정
  chrome_options = Options()
  chrome_options.add_experimental_option("detach", True)
  chrome_options.add_argument("--incognito")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
  driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

  # url 열기
  url = "https://hsreplay.net/account/login"
  driver.get(url)
  
  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  wait_for_element(driver, By.CLASS_NAME, "promo-button")

  btn = driver.find_element(By.CLASS_NAME,"promo-button")
  btn.click()

  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  wait_for_element(driver, By.CLASS_NAME, "input-block")


  id = driver.find_elements(By.CLASS_NAME, "input-block")[0]
  pyperclip.copy("tlthr763@naver.com") # ID 복사
  id.send_keys(Keys.CONTROL, 'v') # ID 붙여넣기

  pw = driver.find_elements(By.CLASS_NAME, "input-block")[1]
  pyperclip.copy("qwer123657.") # 패스워드 복사
  pw.send_keys(Keys.CONTROL, 'v') # 패스워드 붙여넣기
  
  log_btn = driver.find_element(By.CLASS_NAME, "submit-button")
  log_btn.click()

  return driver, chrome_options

In [ ]:
def open_url(time, rank, server):
  if time == "ALL":
      url = f"https://hsreplay.net/meta/#tab=archetypes"
  elif rank == "ALL" and server == "ALL":
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}"
  elif rank == "ALL":
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&region={server}"
  elif server == "ALL":
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&rankRange={rank}"
  else:
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&rankRange={rank}&region={server}"
  
  return url

In [39]:
# 입력값 함수
time, rank, server = get_user_input()

# 엑셀 생성 함수
create_excel()

# 웹브라우저 진입 함수
driver, chrome_options = driver_in()
sleep(2) # 로그인 후 2초동안 대기

# URL 링크 여는 함수
url = open_url()
driver.get(url)

# 엘리먼트가 나타날 때까지 대기
wait_for_element(driver, By.CLASS_NAME, 'class-box-container')

# 직업 클래스
elements = driver.find_elements(By.CSS_SELECTOR, '.class-box')

# 엑셀 열기
wb = openpyxl.load_workbook('./hs_meta.xlsx')
ws = wb.active

# 데이터를 저장할 리스트 생성
data = []

# 직업별 정보 저장 루프
for element in elements:
    # 직업 가져오기
    hero = element.find_element(By.CSS_SELECTOR, '.box-title').text
    #print(f"직업 {hero}")
    
    # 해당 직업에 대한 덱 유형과 정보 클래스
    deck_elements = element.find_elements(By.CSS_SELECTOR, '.table-row-header')
    stats_elements = element.find_elements(By.CSS_SELECTOR, '.table-cell')


    for deck_element in deck_elements:
      # 덱 유형 가져오기
      type_name = deck_element.find_element(By.CSS_SELECTOR, '.tooltip-wrapper').text

      # 승률, 점유율, 게임 수 가져오기
      win_rate = stats_elements[0].text
      popularity = stats_elements[1].text
      games = stats_elements[2].text
    
      # 데이터를 리스트에 추가
      data.append([hero, type_name, win_rate, popularity, games])

      # stats_elements에서 현재 처리한 스탯 정보를 제거
      stats_elements = stats_elements[3:]

# .xlsx 형식으로 엑셀에 추가
for row in data:
    ws.append(row)

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')

In [40]:
# 창의 너비와 높이 설정 
driver.set_window_size(1592, 1039)

# '매치업' 탭 클릭
matchup_tab = driver.find_element(By.ID, 'tab-matchups')
matchup_tab.click()

# 엑셀 새로운 시트 생성
ws_match = create_sheet(wb)

# 덱 이름을 저장할 리스트 생성
data2 = ["",]

# 덱 이름 출력
arche_elements = driver.find_elements(By.CLASS_NAME, 'matchup-row-header')
for arche in arche_elements:
    try :
      isarche=arche.find_element(By.CLASS_NAME, 'archetype-name')
      deck_type = isarche.text 
      data2.append(deck_type)
    except:
      notarche=arche.find_element(By.TAG_NAME, 'aside')
      deck_type = notarche.text
      data2.append(deck_type)
            
# .xlsx 형식으로 엑셀에 추가
ws_match.append(data2)
data2.pop(0)

# 개별 항목을 다시 추가
for row in data2:
    ws_match.append([row])

# 표 데이터를 저장할 리스트 생성
data3 = []

# 표 클래스 찾기
rate_matrix = driver.find_elements(By.CLASS_NAME, 'matchup-cell')
count = 0
row_index = 2  # B2 셀부터 시작

for rate in rate_matrix:
    rate_text = rate.find_element(By.CLASS_NAME, 'tooltip-wrapper')
    rate_element = rate_text.text

    # rate_element를 적절히 처리하여 데이터 추출
    # 예를 들어, rate_element를 콤마(,)로 분리하여 리스트로 변환
    cell_data = rate_element.split(',')

    # data3 리스트에 데이터 추가
    data3.extend(cell_data)

    # 14개의 데이터가 모일 때마다 엑셀에 행 추가
    if len(data3) == 14:
        # 행, 열 수에 맞게 엑셀에 추가
        for col_index, value in enumerate(data3):
            ws_match.cell(row=row_index, column=col_index + 2, value=value)
        
        data3.clear()  # 데이터를 비워줌
        row_index += 1
        count += 1

    if count == 14:
        break

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')
wb.close()

# 크롬창 종료
driver.close()

In [41]:
# 데이터를 포함한 리스트에서 DataFrame을 생성
df = pd.DataFrame(data, columns=['Hero', 'Deck', 'Winrate', 'Popular', 'Games'])

# DataFrame을 UTF-8-sig 인코딩으로 CSV 파일로 저장
df.to_csv('./hs_meta.csv', encoding='utf-8-sig', index=False)

# Matchups 시트를 UTF-8-sig 인코딩으로 CSV 파일로 저장
save_csv()